In [9]:
import pandas as pd
from io import BytesIO
from PIL import Image
import requests
import os

In [10]:
df = pd.read_parquet("hf://datasets/bghira/photo-concept-bucket/photo-concept-bucket.parquet")

In [3]:
df.head(1)

,id,class_label,type,slug,description,alt,created_at,title,location,tags,main_color,colors,width,height,aspect_ratio,url,cogvlm_caption,megapixels
271389,474,quantum mechanics,photo,black-and-white-car-vehicle-vintage,,Person Fixing Car,2015-05-16T15:42:51.000Z,Person Fixing Car,None,"black and white, car, classic, garage, job, me...","[89, 89, 89]","[#3a3a3a, #8c8c8c, #cacaca]",6000,4000,1.5,https://images.pexels.com/photos/474/black-and...,A person is working underneath a car in a gara...,24.0


In [7]:
index = 0
# i want 200 images
while len(os.listdir('images/training_set')) < 200:
    url = df.iloc[index]['url']
    
    response = requests.get(url)
    image = None
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
    else:
        index += 1
        continue
    
    width, height = image.size
    
    # keep only images >= 2k resolution
    if width < 2560 and height < 1440:
        index += 1
        continue

    # center box on image
    target_width, target_height = 2560, 1440
    left = (width - target_width) // 2
    top = (height - target_height) // 2
    right = left + target_width
    bottom = top + target_height
    crop_box = (left, top, right, bottom)
    cropped_image = image.crop(crop_box)
    
    cropped_image.save(f'images/training_set/image_{index}.jpg')
    index += 1    